# Modeling 

- Further problems -> Thresholding?
- Evalution of normel behaviour

## Isolation Forest
- uses tree-based approach to isolate anomalies after modelling itself on "normal" data in an unsupervised fashion.
https://www.projectpro.io/article/anomaly-detection-using-machine-learning-in-python-with-example/555#mcetoc_1fqgmt45ri


In [ ]:
import 